In [7]:
import win32com.client
import re
from pdfminer.high_level import extract_text
import xlwings as xw
import pandas as pd
from tkinter import messagebox
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller
import time
import pyautogui
import pyperclip
from win32 import win32gui
import sys
import datetime as dt
import shutil
import os
import subprocess
import psutil
import win11toast
import clipboard

from IPython.display import clear_output 

#sys.path.append('C:\python_source')
import NERP_PI_LC
from pkb_selenium import *


def handle_regex_error(regex_message:str, skip_exit=None):
    """
        crawl_pdf_pi 함수에서 사용
        오류발생시 코드를 멈추며, skip_exit에 None외의 값 할당시 멈추지 않고 넘김
        (확인버튼을 누르면 이전 작업 지속)
    """
    result_message = regex_message
    messagebox.showinfo('알림', result_message)
    if skip_exit is not None:
        exit

def crawl_pdf_pi(pdftext, address_master_tb):
    """
        스캔이 아닌 pdf를 읽어들여 사용, L001 MDJ PI에만 사용가능
        pdf형식 바뀌면 관련 로직도 변경해야함
    """
    # 기본정보
    convert_port = {
                "BUENOS AIRES": "ARBUE",
                "HANOI": "VNHAN",
                "HONG KONG": "HKHKG",
                "HA NOI, VIETNAM": "VNHAN",
                "Hong Kong": "HKHKG",
                "SHANGHAI":"CNSHA",
                "INCHEON":"KRICN",
                "HONGKONG" : "HKHKG"
                }
    convert_payment = {'NET 60 DAYS FROM B/L DATE':'BL60'}
    convert_partial = {'Partial Allowed':False}
    convert_tranship = {'Tranship Allowed':False}
    currency_option = ['USD','CNY']

    # PI Name, 거래선코드  →  pi_info
    list_pi_name = re.findall('[MDJ]{3}\D+\d+[-]{1}\d+',pdftext)
    if len(list_pi_name) == 1:
        pi_name = re.findall('[MDJ]{3}\D+\d+[-]{1}\d+',pdftext)[0]
    else:
        handle_regex_error('PI번호를 확인해주세요')

    criteria_name = address_master_tb['PI형식'] == pi_name[:5]+'*'
    if len(set(address_master_tb[criteria_name]['거래선코드'].tolist())) == 1:
        customer_code = int(list(set(address_master_tb[criteria_name]['거래선코드'].tolist()))[0])
    else:
        handle_regex_error('거래선코드를 확인해주세요')
        
    pi_info = [pi_name, customer_code]

    # 날짜  →  date_info 
    convert_month = {'JAN':'01','FEB':'02','MAR':'03','APR':'04','MAY':'05','JUN':'06','JUL':'07','AUG':'08','SEP':'09','OCT':'10','NOV':'11','DEC':'12'}
    if len(re.findall('\d{4}-\d{2}-\d{2}',pdftext)) == 2:
        date_open = str(min(re.findall('\d{4}-\d{2}-\d{2}',pdftext))).replace('-','.')
        date_expiry = str(max(re.findall('\d{4}-\d{2}-\d{2}',pdftext))).replace('-','.')
    else:
        handle_regex_error('개설일자와 만료일자를 확인해주세요')

    if len(re.findall('[UNTIL]{5}\s[0-9]{1,2}\s[A-Z]{3}\s[0-9]{4}',pdftext)) == 1:
        shipdate_original_txt = re.findall('[UNTIL]{5}\s[0-9]{1,2}\s[A-Z]{3}\s[0-9]{4}',pdftext)[0]
        shipdate_splited_txt = shipdate_original_txt.split()
        date_shipdate = f'{shipdate_splited_txt[3]}.{convert_month[shipdate_splited_txt[2]]}.{shipdate_splited_txt[1]}'
    else:
        handle_regex_error('선적마감기한을 확인해주세요')

    date_info = [date_open, date_shipdate, date_expiry]

    # 출도착포트, 포트명(생략), 세부 주소 → port_and_address_txt
    portbox = {} 
    pdftext_filterd_port = pdftext[pdftext.find('PORT OF DESTINATION'):pdftext.find('INSPECTION')]
    for port in convert_port.keys():
        if pdftext_filterd_port.find(port) != -1:
            portbox[pdftext.find(port)] = port
    if len(portbox) == 2:
        port_pol = convert_port[portbox[max(portbox.keys())]] 
        port_pod = convert_port[portbox[min(portbox.keys())]]
        port_for_inco = portbox[min(portbox.keys())]
    else:
        handle_regex_error('인식된 포트를 활인해주세요'+str(portbox))

    address_index = str('1') + str(int(list(set(address_master_tb[criteria_name]['거래선코드'].tolist()))[0])) + port_pol + port_pod

    if len(address_master_tb[address_master_tb['주소 INDEX'] == address_index]['Applicant'].tolist()) != 1:
        handle_regex_error('마스터에 중복주소가 있으니 정비해주세요'+str(address_index))

    port_and_address_txt = [
                            [port_pol,port_pod,
                            [], # Port 풀네임 생략
                            {'applicant':address_master_tb[address_master_tb['주소 INDEX'] == address_index]['Applicant'].tolist()[0],
                        'seller':address_master_tb[address_master_tb['주소 INDEX'] == address_index]['Seller'].tolist()[0],
                        'notify':address_master_tb[address_master_tb['주소 INDEX'] == address_index]['Notify'].tolist()[0],
                        'consignee':address_master_tb[address_master_tb['주소 INDEX'] == address_index]['Consignee'].tolist()[0],
                        'shippingmark':address_master_tb[address_master_tb['주소 INDEX'] == address_index]['Shippingmark'].tolist()[0]}
                            ]
            ]

    # 주요정보  →  main_info [cur, amount_floated, payment_code, inco, inco_text, partial, tranship]
    # Cur
    currency_list= []
    for currency in currency_option:
        currency_list = currency_list + re.findall(currency,pdftext)
    if len(currency_list) == 0:
        handle_regex_error('인식된 통화단위가 없거나 신규 통화단위가 있으니 확인해주세요')
    elif len(set(currency_list)) == 1:
        cur = list(set(currency_list))[0]
    else:
        handle_regex_error('인식된 통화단위가 여러개입니다 확인해주세요')

    # Amount
    amount_floated = []
    amount_txt = re.findall('[0-9,\,]+[\.]{1}[0-9]{1}',pdftext)
    #amount_txt = re.findall('[0-9,\,]{0,9}[\.]{1}[0-9]{1,9}',pdftext)
    if len(amount_txt) == 0:
        handle_regex_error('인식된 금액이 없으니 확인해주세요')
    for txt in amount_txt:
        try:
            amount_floated.append(float(txt.replace(',','').replace(' ','')))
        except:
            handle_regex_error('아이템 및 합계 금액을 확인해주세요 변환오류값:'+str(txt))
    amount_floated = max(amount_floated)

    # Payment
    payment_texted = []
    for txt in convert_payment.keys():
        payment_texted = payment_texted + re.findall(txt,pdftext)
        payment_texted = list(set(payment_texted))
    if len(payment_texted) == 0:
        handle_regex_error('검색된 Payment조건이 없습니다 추가하거나 확인해주세요')
    if len(payment_texted) == 1:
        payment_code = convert_payment[payment_texted[0]]
    else:
        handle_regex_error('Payment조건이 2개이상 검색되었으니 확인해주세요')

    # 인코텀즈 (출도착포트 정보를 사용하므로 유의)
    inco_full = []
    for each in ['EXW', 'FOB', 'CPT', 'CFR', 'CIP', 'CIF', 'DDU', 'DDP']:
        inco_full = inco_full + re.findall(each+'\s{1}'+port_for_inco,pdftext)
    if len(inco_full) == 0:
        handle_regex_error('검색된 인코텀즈가 없거나, 도착지명이 다릅니다')
    else:
        inco = inco_full[0][:3]
        inco_text = inco_full[0][4:]

    # Partial
    partial_txt = re.findall('[Partial]{7}\s{1}[A-Z,a-z]{0,9}',pdftext)
    if len(partial_txt) == 1:
        try:    
            partial = convert_partial[partial_txt[0]]
        except:
            partial = True
            handle_regex_error('Partial 내용을 확인해주세요', 'skip_exit')
    else:
        partial = False # 문제있는 경우 일단 Partial 불가로 한다
        handle_regex_error('Partial 관련 인식 문제가 있으니 확인해주세요', 'skip_exit')

    tranship_txt = re.findall('[Tranship]{7}\s{1}[A-Z,a-z]{0,9}',pdftext)
    if len(tranship_txt) == 1:
        try:    
            tranship = convert_tranship[tranship_txt[0]]
        except:
            tranship = True
            handle_regex_error('Tranship 내용을 확인해주세요', 'skip_exit')
    else:
        tranship = False # 문제있는 경우 일단 Tranship 불가로 해야하지만, PC사업부용으로 예외세팅
        #handle_regex_error('Tranship 관련 인식 문제가 있으니 확인해주세요', 'skip_exit')

    main_info = [cur, amount_floated, payment_code, inco, inco_text, partial, tranship]
    
    return (pi_info, date_info, main_info, port_and_address_txt)

def replyFinishedByMail(driver_mail, pi_name:str)->None:
    """
        PI요청온 메일에 대해, PI제목을 조건으로 검색하여 전체회신 메일 발송
    """
    driver_mail.implicitly_wait(10)
    driver_mail.switch_to.frame(driver_mail.find_element(By.CSS_SELECTOR, "#pt-mail-iframe"))
    driver_mail.implicitly_wait(10)
    #txt_to_search = '결재통보'
    txt_to_search = pi_name
    box_input(driver_mail, '검색창 입력', By.XPATH, '/html/body/div[1]/div[2]/div/div[2]/div[2]/div[1]/div[1]/div/div[1]/div/fieldset/input', txt_to_search)
    driver_mail.implicitly_wait(10)
    box_input(driver_mail, '제목검색버튼 대신 Enter입력', By.XPATH, '/html/body/div[1]/div[2]/div/div[2]/div[2]/div[1]/div[1]/div/div[1]/div/fieldset/input', Keys.ENTER)
    #button_click(driver_mail, '제목검색버튼 클릭', By.XPATH, '/html/body/div[4]/div[19]/div/ul/li[4]/a')
    driver_mail.implicitly_wait(10)
    button_click(driver_mail, '1번째 메일 클릭', By.XPATH, '/html/body/div[1]/div[2]/div/div[2]/div[3]/div[1]/div[1]/div/div[3]/div/table/tbody/tr[1]/td[4]/span[1]/a')
    driver_mail.implicitly_wait(10)
    button_click(driver_mail, '전체회신 더보기 클릭', By.XPATH, '/html/body/div[1]/div[2]/div/div[2]/div[3]/div[2]/div/div/div[1]/div[1]/div[1]/span[3]/button[2]')
    # 답장대상 꺼진게 있으면 모두 켜기
    for each in range(1,5):
        #addr = '/html/body/div[4]/div[20]/ul/div/button'
        if get_object(driver_mail, '상태확인', By.XPATH,'/html/body/div[4]/div[20]/ul/div/button[' + str(each) + ']').get_attribute("class") == 'target on':
            pass
        else:
            get_object(driver_mail, '상태확인', By.XPATH,'/html/body/div[4]/div[20]/ul/div/button[' + str(each) + ']').click()
    button_click(driver_mail, '확인/전체회신메일쓰기', By.XPATH, '/html/body/div[4]/div[20]/div/button[2]')

    # 윈도우 전환
    time.sleep(2)
    driver_mail.implicitly_wait(10)
    driver_mail.switch_to.window(driver_mail.window_handles[1])

    # 메일 제목 기존 것 지우고 다시 쓰기
    driver_mail.implicitly_wait(10)
    mailsub_before = get_object(driver_mail, '메일제목가져오기', By.CSS_SELECTOR, "#input-title").get_attribute('value')
    print(mailsub_before)
    mailsub_after = mailsub_before.replace('RE:','[완료]')
    box_clear(driver_mail, '메일제목', By.CSS_SELECTOR, "#input-title")
    driver_mail.implicitly_wait(10)
    time.sleep(1)
    box_input(driver_mail, '메일제목', By.CSS_SELECTOR, "#input-title", mailsub_after)
    driver_mail.implicitly_wait(10)
    button_click(driver_mail, 'html전환', By.XPATH, '/html/body/div/div[2]/div[2]/div/form/table/tbody/tr[5]/td/div[2]/div[2]/div[2]/ul/li[2]')

    driver_mail.implicitly_wait(10)
    mailbody_before = get_object(driver_mail, '기존메일본문 가져오기', By.XPATH, '/html/body/div/div[2]/div[2]/div/form/table/tbody/tr[5]/td/div[2]/div[2]/div[2]/div[2]/textarea').get_attribute('value')
    #pi_name = 'MDJPINAME-01'
    regex_txt_delete = '<P><SPAN style="font-size:10pt;">수신<SPAN lang="EN-US">/ </SPAN><!--?xml:namespace prefix = "o" ns = "urn:schemas-microsoft-com:office:office" /--></SPAN></P>'
    regex_txt_insert = f'<P>안녕하세요.</P><P>아래 번호로 등록했습니다.</P><P>{pi_name}</P>'#<P><BR/></P>'
    mailbody_after = mailbody_before.replace(regex_txt_delete, regex_txt_insert)
    driver_mail.implicitly_wait(10)
    box_clear(driver_mail, '기존내용지우기', By.XPATH, '/html/body/div/div[2]/div[2]/div/form/table/tbody/tr[5]/td/div[2]/div[2]/div[2]/div[2]/textarea')
    arguments_input(driver_mail, 'html본문입력', By.XPATH, '/html/body/div/div[2]/div[2]/div/form/table/tbody/tr[5]/td/div[2]/div[2]/div[2]/div[2]/textarea', mailbody_after)
    
    # 불필요하면 하단 2줄 생략(편집탭 클릭)
    driver_mail.implicitly_wait(10)
    button_click(driver_mail, '편집탭클릭', By.XPATH, '/html/body/div/div[2]/div[2]/div/form/table/tbody/tr[5]/td/div[2]/div[2]/div[2]/ul/li[1]/a')

    driver_mail.implicitly_wait(10)
    driver_mail.switch_to.default_content()
    driver_mail.implicitly_wait(10)
    button_click(driver_mail, '발신버튼', By.CSS_SELECTOR,"body > div > div.win-header > div > div > button.button.default.blue.ng-scope")
    driver_mail.implicitly_wait(10)
    button_click(driver_mail, '발신 Yes', By.CSS_SELECTOR,"#sendExtMailRestrainedUserConfirmDialog > div.pop-btn > button.button.default.b > span")

    driver_mail.switch_to.window(driver_mail.window_handles[0])  # 메일쓰기 프레임으로 돌아가기
pass

In [8]:
# SAP 준비
# rotEntry = win32com.client.GetObject("SAPGUI")
# guiApp = rotEntry.GetScriptingEngine
# connection = guiApp.Children(0)
sessions = NERP_PI_LC.check_and_open_sap('SYSTEM,', 'ID', 'PW',3)
session = sessions[1]

# Knox포털 (메일발송용) 준비
# driver_mail = set_browser("MAIL_SITE_URL", headless='Y')
# driver_mail.implicitly_wait(10)
# # if driver_mail.title != 'ACTUAL WINDOW NAME':
# #     box_input(driver_mail, 'ID입력', By.ID, 'USERID', 'ACTUAL ID')
# #     box_input(driver_mail, 'PW입력', By.ID, 'USERPASSWORD', 'ACTUAL PW')
# #     button_click(driver_mail, '로그인', By.ID, "signIn")
# button_click(driver_mail, '팝업끄기', By.CSS_SELECTOR, "#announce-layer > div.notice-header > button")
# button_click(driver_mail, '메일메뉴가기', By.CSS_SELECTOR,"#menu-4300272")

# 아웃룩 준비
outlook_process_count = 0
for proc in psutil.process_iter():
    if 'OUTLOOK.EXE' in str(proc.name()):
        outlook_process_count += 1
if outlook_process_count == 0:
    subprocess.Popen("C:\\Program Files\\Microsoft Office\\root\\Office16\\OUTLOOK.EXE",)
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
inboxfolder = outlook.GetDefaultFolder(6)
messages = inboxfolder.Items.Restrict("[UnRead] = true")

# 주소 마스터 엑셀 준비
wb = xw.Book('C:\\python_source\\' + 'pi_address.xlsx')
sht = wb.sheets[0]
range_df = sht.range('A2:BL'+str(sht.used_range.last_cell.row))
address_master_tb = sht.used_range.options(pd.DataFrame,index=0).value
wb.close()

# 기준값 세팅
file_path = "C:\\Users\\USER\\Downloads\\"
copy_path_all = ['D:\\()PI보관\\']
time_sleep = 30
mail_sender_name = '담당자이름' # 특정 담당자 이름

list_pi_exist = []
list_mail_exist = []
pi_name = True
count_for_sap = 0

while True:
    for ms in messages:
        if ms.SenderName == mail_sender_name and not('FW:' in ms.Subject):
            print(ms.Subject) # 확인용
            
            if ms.Subject in list_mail_exist:
                pass
            else:
                for file in ms.Attachments:
                    if '.pdf' in file.filename:
                        pi_name = re.findall('MDJ\D+\d+[-]{1}\d+\)',file.filename)[0].replace(')','')
                        pi_file_name = file.filename
                        if pi_name in list_pi_exist:
                            pass
                        else:
                            if NERP_PI_LC.chk_exist_pi_lc(session, pi_name):
                                list_pi_exist.append(pi_name)
                            else:
                                file.SaveAsFile(file_path+file.filename)
                                pdftext = extract_text(file_path + pi_file_name)

                                pi_info, date_info, main_info, port_and_address_txt= crawl_pdf_pi(pdftext, address_master_tb)

                                NERP_PI_LC.input_pi_lc(session, lc_org=1,
                                pi_info=pi_info,#['PI NAME','CUSTOMER CODE']
                    
                                date=date_info,#['2023.01.01','2023.01.15','2023.01.15'], # Open/Last/Expiry date
                                main_info=main_info,#['USD',999999,'BL60','CIP','HANOI',True, False], # CUR, AMOUNT, PAYMENT, INCO, INCOTEXT, PARTIAL, TRANSHIP
                                port_and_address_txt =port_and_address_txt
                                )
                                for copy_path in copy_path_all:
                                    file_from = file_path + pi_file_name
                                    file_to = copy_path + pi_name + '.pdf'
                                    if not os.path.exists(file_to):
                                        shutil.copy(file_from,file_to)
                                        
                                time.sleep(1)
                                #replyFinishedByMail(driver_mail, pi_name) # 기존 메일에 회신
                                win11toast.notify(title= (pi_name + ' / PI등록완료'), body='등록된 PI 확인 후 사업부에 등록메일 회신 필요!',on_click=clipboard.copy(pi_name))
                                #knox_mail_send_html(driver_mail, pi_name + ' / PI등록완료', '등록된 PI 확인 후 사업부에 등록메일 회신 필요!', mail_address=None, attach=None) # 나에게만 메일 알림
            list_mail_exist.append(ms.Subject)
            clear_output()
        ms.unread = False # 모든 작업을 했으면 읽음처리하여 리스트에서 제외
    # 아이템을 다 확인했으면 다시 메일함 확인 후 재시작
    outlook.SendAndReceive(False)
    messages = inboxfolder.Items.Restrict("[UnRead] = true")
    # 프로세스 보존용 (일정시간 미사용시 창이 꺼지므로 주기적으로 사용)
    NERP_PI_LC.loop_tcode(session, 'DUMMY T-CODE')
    #button_click(driver_mail, '메일메뉴가기', By.CSS_SELECTOR,"#menu-4300272")

    time.sleep(5)